In [1]:
import requests
import pandas as pd
from datetime import datetime
from dateutil.relativedelta import relativedelta


In [26]:
# create list of all yyyy-mm for data download (Note, May 2015 is the first month ND data is available in xlxs)
myList = pd.date_range('2015-04-10',datetime.today()- relativedelta(months=2), freq='MS').strftime("%Y_%m").tolist()  # with month in numeric format

# Access the ND DMR data @ https://www.dmr.nd.gov/oilgas/mprindex.asp
website = "https://www.dmr.nd.gov/oilgas/mpr/"

In [ ]:
# Review the list of monthly data
myList

In [28]:
NDMWD = []  #list()  # North Dakota Monthly Well (NDMWD) Data download process ~ 10 minutes
for month in myList:
    target_url = website + month + '.xlsx'
    print(target_url)
    r = requests.get(target_url)
    wb = pd.read_excel(r.content)
    NDMWD.append(wb)

NDMWD = pd.concat(NDMWD)

https://www.dmr.nd.gov/oilgas/mpr/2015_05.xlsx
https://www.dmr.nd.gov/oilgas/mpr/2015_06.xlsx
https://www.dmr.nd.gov/oilgas/mpr/2015_07.xlsx
https://www.dmr.nd.gov/oilgas/mpr/2015_08.xlsx
https://www.dmr.nd.gov/oilgas/mpr/2015_09.xlsx
https://www.dmr.nd.gov/oilgas/mpr/2015_10.xlsx
https://www.dmr.nd.gov/oilgas/mpr/2015_11.xlsx
https://www.dmr.nd.gov/oilgas/mpr/2015_12.xlsx
https://www.dmr.nd.gov/oilgas/mpr/2016_01.xlsx
https://www.dmr.nd.gov/oilgas/mpr/2016_02.xlsx
https://www.dmr.nd.gov/oilgas/mpr/2016_03.xlsx
https://www.dmr.nd.gov/oilgas/mpr/2016_04.xlsx
https://www.dmr.nd.gov/oilgas/mpr/2016_05.xlsx
https://www.dmr.nd.gov/oilgas/mpr/2016_06.xlsx
https://www.dmr.nd.gov/oilgas/mpr/2016_07.xlsx
https://www.dmr.nd.gov/oilgas/mpr/2016_08.xlsx
https://www.dmr.nd.gov/oilgas/mpr/2016_09.xlsx
https://www.dmr.nd.gov/oilgas/mpr/2016_10.xlsx
https://www.dmr.nd.gov/oilgas/mpr/2016_11.xlsx
https://www.dmr.nd.gov/oilgas/mpr/2016_12.xlsx
https://www.dmr.nd.gov/oilgas/mpr/2017_01.xlsx
https://www.d

In [29]:
#Create ND monthly well data table in csv format, which can be downloaded from folder on left hand side
NDMWD.to_csv('NDMWD.csv', index=False)

In [30]:
NDMWD.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1706085 entries, 0 to 20182
Data columns (total 21 columns):
 #   Column      Dtype         
---  ------      -----         
 0   ReportDate  datetime64[ns]
 1   API_WELLNO  int64         
 2   FileNo      int64         
 3   Company     object        
 4   WellName    object        
 5   Quarter     object        
 6   Section     float64       
 7   Township    float64       
 8   Range       float64       
 9   County      object        
 10  FieldName   object        
 11  Pool        object        
 12  Oil         float64       
 13  Wtr         float64       
 14  Days        float64       
 15  Runs        int64         
 16  Gas         float64       
 17  GasSold     int64         
 18  Flared      float64       
 19  Lat         float64       
 20  Long        float64       
dtypes: datetime64[ns](1), float64(10), int64(4), object(6)
memory usage: 286.4+ MB


# **Extract first month oil and gas produciton**

In [31]:
#NDMWD = pd.read_csv('NDMWD.csv')
NDMWD.drop_duplicates(inplace=True)
NDMWD = NDMWD[NDMWD['Days'] >= 25].reset_index(drop=True)
NDMWD['ReportDate'] = pd.to_datetime(NDMWD['ReportDate'], errors = 'coerce')
NDMWD.tail(2)

,ReportDate,API_WELLNO,FileNo,Company,WellName,Quarter,Section,Township,Range,County,...,Pool,Oil,Wtr,Days,Runs,Gas,GasSold,Flared,Lat,Long
1234977,2023-06-01,33009012640000,6347,EMPIRE NORTH DAKOTA LLC,RICE 2,SENW,26.0,163.0,79.0,BOT,...,SPEARFISH/MADISON,25.0,268.0,30.0,0,0.0,0,0.0,48.917191,-100.876462
1234978,2023-06-01,33009019570000,13685,EMPIRE NORTH DAKOTA LLC,RICE 3,NENW,26.0,163.0,79.0,BOT,...,SPEARFISH/MADISON,51.0,6565.0,30.0,0,0.0,0,0.0,48.920808,-100.877109


In [32]:
ffmWells = NDMWD[['API_WELLNO', 'Pool','ReportDate']].groupby('API_WELLNO').min().rename(columns={'ReportDate':'ffMonth'}).reset_index()
ffmWells.tail(3)

,API_WELLNO,Pool,ffMonth
20439,33105058760000,BAKKEN,2023-05-01
20440,33105061150000,MADISON,2023-06-01
20441,33105903870000,DAKOTA,2018-01-01


In [33]:
ffmWells.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20442 entries, 0 to 20441
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   API_WELLNO  20442 non-null  int64         
 1   Pool        20442 non-null  object        
 2   ffMonth     20442 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 479.2+ KB


In [34]:
ffmWells = ffmWells.merge(NDMWD[['API_WELLNO','Pool','ReportDate','Oil','Gas','Wtr','Days','Runs', 'Lat', 'Long']], how='left', left_on=['API_WELLNO','Pool','ffMonth'], right_on = ['API_WELLNO','Pool','ReportDate'])
ffmWells.dropna(inplace=True)
ffmWells.reset_index(drop=True)
ffmWells.tail()

,API_WELLNO,Pool,ffMonth,ReportDate,Oil,Gas,Wtr,Days,Runs,Lat,Long
20437,33105058570000,BAKKEN,2023-05-01,2023-05-01,26701.0,24497.0,64246.0,31.0,26751.0,48.167755,-103.719029
20438,33105058750000,BAKKEN,2023-03-01,2023-03-01,40250.0,31819.0,93444.0,25.0,39436.0,48.169415,-103.699984
20439,33105058760000,BAKKEN,2023-05-01,2023-05-01,31521.0,26441.0,69949.0,31.0,31478.0,48.169418,-103.700107
20440,33105061150000,MADISON,2023-06-01,2023-06-01,938.0,27.0,686.0,30.0,1282.0,48.016557,-103.859187
20441,33105903870000,DAKOTA,2018-01-01,2018-01-01,410.0,0.0,0.0,31.0,946.0,48.404169,-103.475385


In [35]:
ffmWells.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20412 entries, 0 to 20441
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   API_WELLNO  20412 non-null  int64         
 1   Pool        20412 non-null  object        
 2   ffMonth     20412 non-null  datetime64[ns]
 3   ReportDate  20412 non-null  datetime64[ns]
 4   Oil         20412 non-null  float64       
 5   Gas         20412 non-null  float64       
 6   Wtr         20412 non-null  float64       
 7   Days        20412 non-null  float64       
 8   Runs        20412 non-null  float64       
 9   Lat         20412 non-null  float64       
 10  Long        20412 non-null  float64       
dtypes: datetime64[ns](2), float64(7), int64(1), object(1)
memory usage: 1.9+ MB


In [36]:
ffmWells.drop('ReportDate', axis=1, inplace=True )
ffmWells['bpd'] =  ffmWells.Oil/ffmWells.Days
ffmWells['Mcfpd'] =  ffmWells.Gas/ffmWells.Days

In [37]:
ffmWells.tail()

,API_WELLNO,Pool,ffMonth,Oil,Gas,Wtr,Days,Runs,Lat,Long,bpd,Mcfpd
20437,33105058570000,BAKKEN,2023-05-01,26701.0,24497.0,64246.0,31.0,26751.0,48.167755,-103.719029,861.322581,790.225806
20438,33105058750000,BAKKEN,2023-03-01,40250.0,31819.0,93444.0,25.0,39436.0,48.169415,-103.699984,1610.000000,1272.760000
20439,33105058760000,BAKKEN,2023-05-01,31521.0,26441.0,69949.0,31.0,31478.0,48.169418,-103.700107,1016.806452,852.935484
20440,33105061150000,MADISON,2023-06-01,938.0,27.0,686.0,30.0,1282.0,48.016557,-103.859187,31.266667,0.900000
20441,33105903870000,DAKOTA,2018-01-01,410.0,0.0,0.0,31.0,946.0,48.404169,-103.475385,13.225806,0.000000


In [38]:
ffmWells.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20412 entries, 0 to 20441
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   API_WELLNO  20412 non-null  int64         
 1   Pool        20412 non-null  object        
 2   ffMonth     20412 non-null  datetime64[ns]
 3   Oil         20412 non-null  float64       
 4   Gas         20412 non-null  float64       
 5   Wtr         20412 non-null  float64       
 6   Days        20412 non-null  float64       
 7   Runs        20412 non-null  float64       
 8   Lat         20412 non-null  float64       
 9   Long        20412 non-null  float64       
 10  bpd         20412 non-null  float64       
 11  Mcfpd       20412 non-null  float64       
dtypes: datetime64[ns](1), float64(9), int64(1), object(1)
memory usage: 2.0+ MB


# **Wells to MHB allocation**

In [39]:
# Maidenhead grid locators courtesy of Walter Underwood K6WRU:
# https://ham.stackexchange.com/questions/221/how-can-one-convert-from-lat-long-to-grid-square


In [40]:
upper = 'ABCDEFGHIJKLMNOPQRSTUVWX'
lower = 'abcdefghijklmnopqrstuvwx'

In [41]:
def gridCalc(row):
    dec_lat = row['Lat']
    dec_lon = row['Long']
    if not (-180<=dec_lon<180):
        sys.stderr.write('longitude must be -180<=lon<180, given %f\n'%dec_lon)
        sys.exit(32)
    if not (-90<=dec_lat<90):
        sys.stderr.write('latitude must be -90<=lat<90, given %f\n'%dec_lat)
        sys.exit(33) # can't handle north pole, sorry, [A-R]

    adj_lat = dec_lat + 90.0
    adj_lon = dec_lon + 180.0

    grid_lat_sq = upper[int(adj_lat/10)];
    grid_lon_sq = upper[int(adj_lon/20)];

    grid_lat_field = str(int(adj_lat%10))
    grid_lon_field = str(int((adj_lon/2)%10))

    adj_lat_remainder = (adj_lat - int(adj_lat)) * 60
    adj_lon_remainder = ((adj_lon) - int(adj_lon/2)*2) * 60

    grid_lat_subsq = lower[int(adj_lat_remainder/2.5)]
    grid_lon_subsq = lower[int(adj_lon_remainder/5)]

    return grid_lon_sq + grid_lat_sq + grid_lon_field + grid_lat_field + grid_lon_subsq + grid_lat_subsq

In [42]:
ffmWells['MHB'] = ffmWells.apply(gridCalc, axis=1)

In [43]:
ffmWells.tail()

,API_WELLNO,Pool,ffMonth,Oil,Gas,Wtr,Days,Runs,Lat,Long,bpd,Mcfpd,MHB
20437,33105058570000,BAKKEN,2023-05-01,26701.0,24497.0,64246.0,31.0,26751.0,48.167755,-103.719029,861.322581,790.225806,DN88de
20438,33105058750000,BAKKEN,2023-03-01,40250.0,31819.0,93444.0,25.0,39436.0,48.169415,-103.699984,1610.000000,1272.760000,DN88de
20439,33105058760000,BAKKEN,2023-05-01,31521.0,26441.0,69949.0,31.0,31478.0,48.169418,-103.700107,1016.806452,852.935484,DN88de
20440,33105061150000,MADISON,2023-06-01,938.0,27.0,686.0,30.0,1282.0,48.016557,-103.859187,31.266667,0.900000,DN88ba
20441,33105903870000,DAKOTA,2018-01-01,410.0,0.0,0.0,31.0,946.0,48.404169,-103.475385,13.225806,0.000000,DN88gj


In [44]:
ffmWells.to_csv('ffmWells.csv', index=False)